In [1]:
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn.utils import spectral_norm as SN
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from torchvision.utils import make_grid, save_image

In [2]:
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:
    celeba_dataset=datasets.CelebA(root="C:/Users/maddo/Downloads/COdes from Work/Working on now",split='train',target_type='attr',transform=transform,download=True)

Files already downloaded and verified


In [4]:
train_loader=DataLoader(celeba_dataset, batch_size=40, shuffle=True)

In [5]:
class generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.cn1=nn.ConvTranspose2d(in_channels=512,out_channels=256,stride=2,padding=1,kernel_size=4)   #256 layers 2x2
        self.cn2=nn.ConvTranspose2d(in_channels=256,out_channels=128,stride=2,padding=1,kernel_size=4)   #128 layers 4x4
        self.cn3=nn.ConvTranspose2d(in_channels=128,out_channels=64,stride=2,padding=1,kernel_size=4)  #200 layers 8x8
        self.cn4=nn.ConvTranspose2d(in_channels=64,out_channels=32,stride=2,padding=1,kernel_size=4)  #200 layers 16x16
        self.cn5=nn.ConvTranspose2d(in_channels=32,out_channels=16,stride=2,padding=1,kernel_size=4)   #256 layers 32x32
        self.cn7=nn.ConvTranspose2d(in_channels=16,out_channels=3,stride=2,padding=1,kernel_size=4) #132 layers 128x128  output is 128
        self.leakyrelu=nn.LeakyReLU(negative_slope=0.1)
        self.tan=nn.Tanh()
    def forward(self,x):
        x=self.leakyrelu(self.cn1(x))
        x=self.leakyrelu(self.cn2(x))
        x=self.leakyrelu(self.cn3(x))
        x=self.leakyrelu(self.cn4(x))
        x=self.leakyrelu(self.cn5(x))
        x=self.leakyrelu(self.cn7(x))
        return self.tan(x)

In [6]:
class critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            SN(nn.Conv2d(3, 64, 4, 2, 1)),   # 128x128 -> 64x64
            nn.LeakyReLU(0.2, inplace=True),
            SN(nn.Conv2d(64, 128, 4, 2, 1)), # 64x64 -> 32x32
            nn.LeakyReLU(0.2, inplace=True),
            SN(nn.Conv2d(128, 256, 4, 2, 1)),# 32x32 -> 16x16
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1),# 16x16 -> 8x8
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(512*8*8, 1)
        )

    def forward(self, x):
        return self.net(x)


In [7]:
def gradientpenalty(critic,real,fake,lamb):
    epsilon=torch.rand(real.size(0),1,1,1)
    epsilon=epsilon.to('cuda')
    epsilon=epsilon.expand_as(real)
    interm=epsilon*real+(1-epsilon)*fake
    interm.requires_grad_(True)
    d_inter=critic(interm)
    grad=torch.autograd.grad(outputs=d_inter,inputs=interm,create_graph=True,grad_outputs=torch.ones_like(d_inter))[0]
    grad_norm=torch.sqrt(torch.sum(grad**2,dim=[1,2,3])+1e-12)
    penalty=((grad_norm-1)**2).mean()*lamb
    return penalty

In [8]:
enc=generator().to('cuda')
crit=critic().to('cuda')

genoptim=optim.Adam(enc.parameters(),lr=5e-5,betas=(0.5,0.99))
criticoptim=optim.Adam(crit.parameters(),lr=2e-5,betas=(0.5,0.99))
lambd=15
epochs=50

In [9]:
fixed_z = torch.randn(64, 512, 2, 2, device='cuda') 

In [ ]:
logevent=100
for i in range(epochs):
    step=0
    for x,y in train_loader:
        z=torch.randn(x.size(0),512,2,2).to('cuda')
        
        fake1=enc(z).detach()
        
        lossf=crit(fake1).mean()
        x=x.to('cuda')
        lossreal=crit(x).mean()
        penal=gradientpenalty(crit,x,fake1,lambd)
        loss_tot=lossf-lossreal+penal
        criticoptim.zero_grad(set_to_none=True)
        loss_tot.backward()
        criticoptim.step()
    
        z2=torch.randn(x.size(0),512,2,2).to('cuda')
        fake2=enc(z2)
        lossg=-crit(fake2).mean()
        genoptim.zero_grad(set_to_none=True)
        lossg.backward()
        genoptim.step()
        step+=1
        if step%logevent==0:
            print(f" For epoch number {i} loss real is {loss_tot} , penalty is {penal} ,loss gen is {lossg}")
    
    enc.eval()
    with torch.no_grad():
        fake_eval = enc(fixed_z).clamp(-1, 1)      # in [-1,1]
        imgs = (fake_eval + 1) / 2                 # to [0,1] for saving
        grid = make_grid(imgs, nrow=8, padding=2)
        save_image(grid, f"samples/epoch_{i:04d}.png")
    enc.train()

    

C:\Users\maddo\anaconda3\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


 For epoch number 0 loss real is 14.166132926940918 , penalty is 14.161747932434082 ,loss gen is 0.011434468440711498
 For epoch number 0 loss real is 14.251181602478027 , penalty is 14.276069641113281 ,loss gen is 0.010483688674867153
 For epoch number 0 loss real is 14.235743522644043 , penalty is 14.287333488464355 ,loss gen is 0.009915899485349655
 For epoch number 0 loss real is 14.215044975280762 , penalty is 14.286346435546875 ,loss gen is 0.009648767299950123
 For epoch number 0 loss real is 14.178375244140625 , penalty is 14.279899597167969 ,loss gen is 0.009451662190258503
 For epoch number 0 loss real is 14.154330253601074 , penalty is 14.27449893951416 ,loss gen is 0.008914061821997166
 For epoch number 0 loss real is 14.098526954650879 , penalty is 14.256949424743652 ,loss gen is 0.008564875461161137
 For epoch number 0 loss real is 14.043619155883789 , penalty is 14.237499237060547 ,loss gen is 0.008039471693336964
 For epoch number 0 loss real is 14.000515937805176 , pen

In [ ]:
try1=torch.randn(2,512,2,2).to('cuda')